In [7]:
import numpy as np

# Inversión sísmica basada en modelo

In [8]:
def calc_r(m0, j=999):
    
    if j != 999:
        r = (m0[j+1] - m0[j])/(m0[j+1] + m0[j])
    else:
        r = []
        for i in range(1, len(m0)):

            num = m0[i] - m0[i-1]
            den = m0[i] + m0[i-1]
            r_temp = num/den
            r.append(r_temp)
        r = np.array(r)
    
    return r

In [9]:
def calc_jacobian(m0, r0):
    
    G = np.zeros([len(m0), len(r0)])
    
    for i in range(len(m0)): # for each Z
        
        # change the model
        dZ = m0[i]*0.05 # dZ
        m_temp = m0.copy()
        m_temp[i] += dZ

        for j in range(len(r0)): # for each r
            
            save = False
            
            if i == j:
                save = True
                j = i
                
            elif i == (j+1):
                j = i-1
                save = True
                
            if save:

                r_new = calc_r(m_temp, j)
                dr =  r0[j] - r_new # dr
                G[i][j] = dr/dZ # update jacobian matrix
                

                
    return G.T

In [10]:
def update_m(d_obs, d_est, m0):
    
    G = calc_jacobian(m0, d_est) # jacobian matrix
    G_square = np.matmul(G.T, G) # square jacobian for inversion
    G_inverse = np.linalg.pinv(G_square)
    E = (d_obs - d_est).reshape(3,1)
    produ = np.matmul(G_inverse, G.T)
    dm = np.matmul(produ, E)
    update_m = m0 + dm
    
    return update_m
    

In [11]:
def L1_error(y_true, y_pred):
    n = len(y_true)
    # Calculate the absolute percentage error for each data point
    abs_percentage_error = np.abs((y_pred - y_true) / y_true)
    
    # Calculate the mean absolute percentage error as a percentage of n
    L1 = (np.sum(abs_percentage_error) / n) * 100
    #print(f"L1 error: {L1:.2f}%")
    return L1

def L2_error(y_true, y_pred):
    n = len(y_true)
    # Calculate the squared percentage error for each data point
    squared_percentage_error = ((y_true - y_pred) / y_true) ** 2
    
    # Calculate the mean squared percentage error and take the square root as a percentage of n
    L2 = (np.sqrt(np.sum(squared_percentage_error)) / n) * 100
    #print(f"L2 error: {L2:.2f}%")
    return L2

In [12]:
def iterate_inv(m0, d_obs, d_est, iter_=20):
    
    m_est = m0
    
    for i in range(iter_):
        print('='*50, i, '='*50)
        L1 = L1_error(d_obs, d_est)
        L2 = L2_error(d_obs, d_est)
        print(f'd_obs: {d_obs}')
        print(f'd_est: {d_est}')
        print(f'L1: {L1}')
        print(f'L2: {L2}')
        m_est = update_m(d_obs, d_est, m_est) # Z estimado
        d_est = calc_r(m_est, j=999) # r estimado
        
        
    return d_est, m_est

In [13]:
# model 
m0 = np.array([1.3, 1.4, 1.2, 1.1]).reshape(4,1) # model [MPa]
r0 = calc_r(m0).reshape(3,1) # synthetic reflectivity profile [seismogram] - Datos estimados
r = np.array([0.01, -0.012, -0.011]).reshape(3,1) # real reflectivity profile (from seismic data) [seismogram] - Datos observados

In [14]:
m0

array([[1.3],
       [1.4],
       [1.2],
       [1.1]])

In [15]:
r0

array([[ 0.03703704],
       [-0.07692308],
       [-0.04347826]])

In [16]:
r

array([[ 0.01 ],
       [-0.012],
       [-0.011]])

In [17]:
J = calc_jacobian(m0, r0)
J

array([[ 0.3750586 , -0.34764006,  0.        ,  0.        ],
       [ 0.        ,  0.34572169, -0.4048583 ,  0.        ],
       [ 0.        ,  0.        ,  0.40530582, -0.44309056]])

In [18]:
new_Z, new_r = iterate_inv(m0=m0, d_obs=r, d_est=r0, iter_=20)

================================================== 0 ==================================================
d_obs: [[ 0.01 ]
 [-0.012]
 [-0.011]]
d_est: [[ 0.03703704]
 [-0.07692308]
 [-0.04347826]]
L1: 368.8843094640192
L2: 224.3473320187614
================================================== 1 ==================================================
d_obs: [[ 0.01 ]
 [-0.012]
 [-0.011]]
d_est: [[ 0.06330079]
 [-0.14206387]
 [-0.07964164]]
L1: 746.96279529256
L2: 453.1686782623698
================================================== 2 ==================================================
d_obs: [[ 0.01 ]
 [-0.012]
 [-0.011]]
d_est: [[ 0.11326129]
 [-0.27139106]
 [-0.16594475]]
L1: 1534.2645889628845
L2: 926.3364067735829
================================================== 3 ==================================================
d_obs: [[ 0.01 ]
 [-0.012]
 [-0.011]]
d_est: [[ 0.20754983]
 [-0.53360935]
 [-0.47126065]]
L1: 3502.143540754667
L2: 2116.1865356840276
============================================